
## Use Case 2: Personalised Marketing for Credit Card Offers

**Objective:** Segment customers based on spending patterns and demographics to optimize targeted credit card marketing campaigns.

**Dataset:** `BankChurners.csv`

**Key Features:**
- Income level
- Average monthly spending
- Existing credit cards
- Customer response to previous offers

In [0]:
# Use Case 2: Data Loading
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler

df_marketing = spark.read.csv('/Volumes/workspace/default/capstone-project/BankChurners.csv', 
                               header=True, inferSchema=True)

print("Dataset loaded successfully!")
print(f"Total Records: {df_marketing.count()}")
print(f"Total Columns: {len(df_marketing.columns)}")

display(df_marketing.limit(10))


Dataset loaded successfully!
Total Records: 3000
Total Columns: 7


Customer_ID,Income,Avg_Monthly_Spend,Existing_Cards,Age,Credit_Score,Response_to_Offer
20000,133043,2196.19,4,45,403,0
20001,51028,4506.17,4,57,670,0
20002,120168,8447.15,2,21,460,0
20003,144687,13664.4,2,41,687,1
20004,161538,4893.43,3,61,711,0
20005,171900,3388.8,1,59,353,0
20006,180469,7440.5,1,61,751,1
20007,146387,2771.12,3,46,432,0
20008,163744,4439.52,4,62,445,0
20009,90676,14806.15,2,23,430,0


In [0]:
# Use Case 2: Data Cleaning & Preprocessing
from pyspark.sql.functions import col, when, isnan, count

# Check for missing values
print("=== Missing Values Analysis ===")
df_marketing.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df_marketing.columns]).show()

# Display basic statistics
print("\n=== Dataset Statistics ===")
df_marketing.describe().show()

# Remove duplicates
df_marketing = df_marketing.dropDuplicates()
print(f"\nRecords after removing duplicates: {df_marketing.count()}")

# Drop unnecessary columns (Customer_ID is not needed for analysis)
if 'Customer_ID' in df_marketing.columns:
    df_marketing = df_marketing.drop('Customer_ID')

# Handle categorical variables - get column types
print("\n=== Column Data Types ===")
df_marketing.printSchema()

# Select relevant features based on ACTUAL column names
existing_features = [
    'Income',
    'Avg_Monthly_Spend',
    'Existing_Cards',
    'Age',
    'Credit_Score'
]

print(f"\n=== Selected Features for Analysis: {len(existing_features)} ===")
print(existing_features)

# Handle missing values - fill with median for numeric columns
for column in existing_features:
    median_val = df_marketing.approxQuantile(column, [0.5], 0.01)[0]
    df_marketing = df_marketing.withColumn(
        column,
        when(col(column).isNull(), median_val).otherwise(col(column))
    )

# Display cleaned data
print("\n=== Cleaned Dataset Sample ===")
display(df_marketing.select(existing_features).limit(10))

print("\n✅ Data cleaning completed successfully!")
print(f"Final dataset shape: {df_marketing.count()} rows")


=== Missing Values Analysis ===
+-----------+------+-----------------+--------------+---+------------+-----------------+
|Customer_ID|Income|Avg_Monthly_Spend|Existing_Cards|Age|Credit_Score|Response_to_Offer|
+-----------+------+-----------------+--------------+---+------------+-----------------+
|          0|     0|                0|             0|  0|           0|                0|
+-----------+------+-----------------+--------------+---+------------+-----------------+


=== Dataset Statistics ===
+-------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------+
|summary|      Customer_ID|           Income|Avg_Monthly_Spend|    Existing_Cards|               Age|     Credit_Score|  Response_to_Offer|
+-------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-------------------+
|  count|             3000|             3000|             3000|           

Income,Avg_Monthly_Spend,Existing_Cards,Age,Credit_Score
141988.0,14323.26,4.0,49.0,347.0
144614.0,7297.39,3.0,63.0,322.0
147406.0,10520.6,4.0,53.0,773.0
72960.0,14611.18,1.0,47.0,524.0
178464.0,1977.27,2.0,58.0,641.0
185689.0,8217.46,3.0,49.0,818.0
106623.0,3453.34,1.0,62.0,545.0
21400.0,8152.0,4.0,30.0,448.0
154091.0,4546.51,4.0,42.0,442.0
81405.0,9997.59,2.0,50.0,790.0



✅ Data cleaning completed successfully!
Final dataset shape: 3000 rows


In [0]:
# Use Case 2: Feature Engineering & Vectorization
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Assemble features into a vector
assembler = VectorAssembler(
    inputCols=existing_features,
    outputCol="features_raw",
    handleInvalid="skip"
)

df_assembled = assembler.transform(df_marketing)

# Standardize features for better clustering
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

print("✅ Feature engineering completed!")
print(f"Total records with features: {df_scaled.count()}")
display(df_scaled.select("features", "Income", "Avg_Monthly_Spend").limit(5))


✅ Feature engineering completed!
Total records with features: 3000


features,Income,Avg_Monthly_Spend
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.5963534902115869"",""1.595532783165085"",""1.4120997377993534"",""0.49499970466925464"",""-1.3957574732024107""]}",141988.0,14323.26
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.6467829800345717"",""-0.10964924930512947"",""0.7065202552213962"",""1.6135829176311045"",""-1.5511934979965833""]}",144614.0,7297.39
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.7004003203185676"",""0.6726253699062024"",""1.4120997377993534"",""0.8145949083726403"",""1.2528723892902902""]}",147406.0,10520.6
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""-0.7292545918527669"",""1.6654111058196208"",""-0.7046387099345183"",""0.3352021028175618"",""-0.29527041765966877""]}",72960.0,14611.18
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.2968356192227035"",""-1.4008449353319758"",""9.40772643438896E-4"",""1.2140889130018724"",""0.4321701783770589""]}",178464.0,1977.27


Databricks visualization. Run in Databricks to view.

In [0]:
# Use Case 2: Customer Segmentation using K-Means
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Train K-Means model with 4 customer segments
kmeans = KMeans(
    featuresCol="features",
    predictionCol="segment",
    k=4,
    seed=42,
    maxIter=20
)

print("Training K-Means clustering model...")
kmeans_model = kmeans.fit(df_scaled)

# Make predictions
df_segmented = kmeans_model.transform(df_scaled)

# Evaluate clustering
evaluator = ClusteringEvaluator(
    featuresCol="features",
    predictionCol="segment",
    metricName="silhouette"
)

silhouette_score = evaluator.evaluate(df_segmented)
print(f"\n✅ Silhouette Score: {silhouette_score:.4f}")

# Show segment distribution
print("\n=== Customer Segment Distribution ===")
df_segmented.groupBy("segment").count().orderBy("segment").show()

print(f"Total customers segmented: {df_segmented.count()}")
display(df_segmented.select("Income", "Avg_Monthly_Spend", "Credit_Score", "segment").limit(20))


Training K-Means clustering model...

✅ Silhouette Score: 0.2409

=== Customer Segment Distribution ===
+-------+-----+
|segment|count|
+-------+-----+
|      0|  827|
|      1|  691|
|      2|  744|
|      3|  738|
+-------+-----+

Total customers segmented: 3000


Income,Avg_Monthly_Spend,Credit_Score,segment
141988.0,14323.26,347.0,0
144614.0,7297.39,322.0,0
147406.0,10520.6,773.0,3
72960.0,14611.18,524.0,3
178464.0,1977.27,641.0,2
185689.0,8217.46,818.0,3
106623.0,3453.34,545.0,1
21400.0,8152.0,448.0,0
154091.0,4546.51,442.0,0
81405.0,9997.59,790.0,3


In [0]:
# Use Case 2: Segment Analysis & Profiling
from pyspark.sql.functions import avg, col, count

# Calculate average characteristics for each segment
print("=== Segment Profiling ===")
segment_profiles = df_segmented.groupBy("segment").agg(
    avg("Income").alias("Avg_Income"),
    avg("Avg_Monthly_Spend").alias("Avg_Monthly_Spend"),
    avg("Credit_Score").alias("Avg_Credit_Score"),
    avg("Existing_Cards").alias("Avg_Existing_Cards"),
    avg("Age").alias("Avg_Age"),
    count("*").alias("Customer_Count")
).orderBy("segment")

display(segment_profiles)

# Segment interpretation
print("\n=== Customer Segment Insights ===")
segment_data = segment_profiles.collect()

for row in segment_data:
    segment_id = row['segment']
    income = row['Avg_Income']
    spend = row['Avg_Monthly_Spend']
    credit = row['Avg_Credit_Score']
    
    print(f"\n📊 Segment {segment_id}:")
    print(f"   • Average Income: ${income:,.2f}")
    print(f"   • Average Monthly Spend: ${spend:,.2f}")
    print(f"   • Average Credit Score: {credit:.0f}")
    print(f"   • Total Customers: {row['Customer_Count']}")

print("\n✅ Segment analysis completed!")


=== Segment Profiling ===


segment,Avg_Income,Avg_Monthly_Spend,Avg_Credit_Score,Avg_Existing_Cards,Avg_Age,Customer_Count
0,101250.67472793229,8344.315610640877,451.8149939540508,3.3143893591293834,41.29746070133011,827
1,118791.84370477569,7437.38460202605,431.72793053545587,0.6599131693198264,43.93921852387844,691
2,112928.34543010753,3664.568104838707,703.5887096774194,1.9408602150537635,43.049731182795696,744
3,112418.18292682926,11492.025203252031,703.2886178861788,1.8360433604336044,43.18428184281843,738



=== Customer Segment Insights ===

📊 Segment 0:
   • Average Income: $101,250.67
   • Average Monthly Spend: $8,344.32
   • Average Credit Score: 452
   • Total Customers: 827

📊 Segment 1:
   • Average Income: $118,791.84
   • Average Monthly Spend: $7,437.38
   • Average Credit Score: 432
   • Total Customers: 691

📊 Segment 2:
   • Average Income: $112,928.35
   • Average Monthly Spend: $3,664.57
   • Average Credit Score: 704
   • Total Customers: 744

📊 Segment 3:
   • Average Income: $112,418.18
   • Average Monthly Spend: $11,492.03
   • Average Credit Score: 703
   • Total Customers: 738

✅ Segment analysis completed!


In [0]:
# Use Case 2: Personalized Credit Card Marketing Recommendations
from pyspark.sql.functions import when, col

# Define marketing strategy for each segment based on income and spending
df_recommendations = df_segmented.withColumn(
    "credit_card_offer",
    when(col("segment") == 0, "Premium Rewards Card - High income, high spending")
    .when(col("segment") == 1, "Cash Back Card - Moderate income, regular spending")
    .when(col("segment") == 2, "Starter Card - Building credit, lower income")
    .when(col("segment") == 3, "Balance Transfer Card - High credit score, strategic users")
    .otherwise("Standard Card")
)

df_recommendations = df_recommendations.withColumn(
    "marketing_channel",
    when(col("segment") == 0, "Personalized email + VIP events")
    .when(col("segment") == 1, "Direct mail + SMS campaigns")
    .when(col("segment") == 2, "Digital ads + mobile app notifications")
    .when(col("segment") == 3, "Financial advisory consultation + email")
    .otherwise("Standard email")
)

print("=== Marketing Campaign Recommendations ===")
display(df_recommendations.select("segment", "credit_card_offer", "marketing_channel", 
                                   "Income", "Avg_Monthly_Spend", "Credit_Score").limit(20))

# Summary statistics
print("\n=== Campaign Distribution ===")
df_recommendations.groupBy("segment", "credit_card_offer").count().orderBy("segment").show(truncate=False)

print("\n✅ Marketing recommendations generated successfully!")
print(f"Total customers targeted: {df_recommendations.count()}")


=== Marketing Campaign Recommendations ===


segment,credit_card_offer,marketing_channel,Income,Avg_Monthly_Spend,Credit_Score
0,"Premium Rewards Card - High income, high spending",Personalized email + VIP events,141988.0,14323.26,347.0
0,"Premium Rewards Card - High income, high spending",Personalized email + VIP events,144614.0,7297.39,322.0
3,"Balance Transfer Card - High credit score, strategic users",Financial advisory consultation + email,147406.0,10520.6,773.0
3,"Balance Transfer Card - High credit score, strategic users",Financial advisory consultation + email,72960.0,14611.18,524.0
2,"Starter Card - Building credit, lower income",Digital ads + mobile app notifications,178464.0,1977.27,641.0
3,"Balance Transfer Card - High credit score, strategic users",Financial advisory consultation + email,185689.0,8217.46,818.0
1,"Cash Back Card - Moderate income, regular spending",Direct mail + SMS campaigns,106623.0,3453.34,545.0
0,"Premium Rewards Card - High income, high spending",Personalized email + VIP events,21400.0,8152.0,448.0
0,"Premium Rewards Card - High income, high spending",Personalized email + VIP events,154091.0,4546.51,442.0
3,"Balance Transfer Card - High credit score, strategic users",Financial advisory consultation + email,81405.0,9997.59,790.0



=== Campaign Distribution ===
+-------+----------------------------------------------------------+-----+
|segment|credit_card_offer                                         |count|
+-------+----------------------------------------------------------+-----+
|0      |Premium Rewards Card - High income, high spending         |827  |
|1      |Cash Back Card - Moderate income, regular spending        |691  |
|2      |Starter Card - Building credit, lower income              |744  |
|3      |Balance Transfer Card - High credit score, strategic users|738  |
+-------+----------------------------------------------------------+-----+


✅ Marketing recommendations generated successfully!
Total customers targeted: 3000
